In [1]:
import itertools
from parlay_system import *
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [2]:
pistons = MoneyLine(event="pistons", bet_amount=100, odds=235)
phi76 = MoneyLine(event="phi76", bet_amount=100, odds=-286)

raptors = MoneyLine(event="raptors", bet_amount=100, odds=180)
pacers = MoneyLine(event="pacers", bet_amount=100, odds=-220)

wizards = MoneyLine(event="wizards", bet_amount=100, odds=132)
knicks = MoneyLine(event="knicks", bet_amount=100, odds=-157)

bulls = MoneyLine(event="bulls", bet_amount=100, odds=163)
magic = MoneyLine(event="magic", bet_amount=100, odds=-195)

hawks = MoneyLine(event="hawks", bet_amount=100, odds=130)
cavs = MoneyLine(event="cavs", bet_amount=100, odds=-159)


binaries = [
#     [pistons, phi76],
#     [raptors, pacers],
    [wizards, knicks],
    [bulls, magic],
    [hawks, cavs]
]

results = [
#     [0, 1], #[pistons, phi76],
#     [0, 1], #[raptors, pacers],
    [0, 1], #[wizards, knicks],
    [0, 1], #[bulls, magic],
    [0, 1], #[hawks, cavs]
]

override_arr = [0, 2, 99]


def create_dicts(in_binaries, in_results):
    ml_lookup = {}
    ml_win_lookup = {}
    id = 0
    for bi in range(len(in_binaries)):
        binary = in_binaries[bi]
        for ml_i in range(len(binary)):
            ml = binary[ml_i]
            ml.set_index(id) 
            is_winner = in_results[bi][ml_i]
            ml_lookup[id] = ml.event
            ml_win_lookup[id] = is_winner
            id += 1
    
    return ml_lookup, ml_win_lookup

ml_lookup, ml_outcome_lookup = create_dicts(binaries, results)

In [3]:
def findsubsets(s, n):
    return list(itertools.combinations(s, n))

TOURNEY_SIZE = len(binaries)
SELECT_NUM = 3
s = set()
n = SELECT_NUM

for i in range(TOURNEY_SIZE):
    s.add(i)

all_subsets = findsubsets(s, n)

In [4]:
len(all_subsets)

1

In [5]:
csv = []
df = pd.DataFrame()

for i in range(len(all_subsets)):
    current_subset = all_subsets[i]
    parlay_binaries = []
    for i in range(len(current_subset)):
        index = current_subset[i]
        parlay_binaries.append(binaries[index])

    ps = ParlaySystem(binaries=parlay_binaries,
                    target_profit=0.15,
                    bounds=(0.01, 30),
                    binary_index_arr=list(current_subset),
                    binary_results_arr=results,
                    index_to_ml=ml_lookup,
                    index_to_outcome=ml_outcome_lookup,
                    override_arr=override_arr
                    )
    
    csv_res, raw_df = ps.slsqp_solver()
    df = pd.concat([df, raw_df])
    csv.append(csv_res)

     fun: -0.015910381635369553
     jac: array([-0.75421   , -0.24238553, -0.00905128,  0.28535881, -0.23773328,
        0.12339918,  0.28803487,  0.49576437])
 message: 'Positive directional derivative for linesearch'
    nfev: 60
     nit: 9
    njev: 5
  status: 8
 success: False
       x: array([0.03366609, 0.04753548, 0.05751858, 0.03796488, 0.04771415,
       0.05034698, 0.03771299, 0.01      ])
slsqp_solver: 
                 event    index[]     result  event_status         odds    bet       mult  payout  profit
7    knicks_magic_cavs  [1, 3, 5]  [1, 1, 1]          True   303.388502  0.010   4.033885  0.0403  -0.282
0  wizards_bulls_hawks  [0, 2, 4]  [0, 0, 0]         False  1303.368000  0.034  14.033680  0.4725   0.150
1   wizards_bulls_cavs  [0, 2, 5]  [0, 0, 1]         False   893.908428  0.048   9.939084  0.4725   0.150
4   knicks_bulls_hawks  [1, 2, 4]  [1, 0, 0]         False   890.186624  0.048   9.901866  0.4725   0.150
2  wizards_magic_hawks  [0, 3, 4]  [0, 1, 0]     

In [6]:
df.describe()

,odds,bet,mult,payout,profit
count,8.000000,8.000000,8.000000,8.000000,8.000000
mean,705.082286,0.040500,8.050823,0.338387,0.016000
std,318.462456,0.014609,3.184625,0.164260,0.164225
min,303.388502,0.010000,4.033885,0.040300,-0.282000
25%,471.177738,0.037000,5.711777,0.216525,-0.105750
50%,654.260841,0.043000,7.542608,0.408700,0.086500
75%,891.117075,0.048500,9.911171,0.472500,0.150000
max,1303.368000,0.058000,14.033680,0.472500,0.150000


In [7]:
negative_df = df.loc[df.profit < 0]
negative_df = negative_df.sort_values(by=['event_status', 'profit'], ascending=[False, True])
negative_df.describe()

,odds,bet,mult,payout,profit
count,3.000000,3.000000,3.000000,3.000000,3.000000
mean,414.891186,0.028667,5.148912,0.157400,-0.165000
std,96.570089,0.016166,0.965701,0.101418,0.101336
min,303.388502,0.010000,4.033885,0.040300,-0.282000
25%,386.480304,0.024000,4.864803,0.127550,-0.195000
50%,469.572105,0.038000,5.695721,0.214800,-0.108000
75%,470.642527,0.038000,5.706425,0.215950,-0.106500
max,471.712950,0.038000,5.717129,0.217100,-0.105000


In [8]:
positive_df = df.loc[df.profit >= 0]
positive_df = positive_df.sort_values(by=['event_status', 'profit'], ascending=[False, False])
positive_df.describe()

,odds,bet,mult,payout,profit
count,5.000000,5.000000,5.000000,5.00000,5.000000
mean,879.196947,0.047600,9.791969,0.44698,0.124600
std,267.892043,0.008649,2.678920,0.05260,0.052439
min,601.280655,0.034000,7.012807,0.35310,0.031000
25%,707.241026,0.048000,8.072410,0.46430,0.142000
50%,890.186624,0.048000,9.901866,0.47250,0.150000
75%,893.908428,0.050000,9.939084,0.47250,0.150000
max,1303.368000,0.058000,14.033680,0.47250,0.150000


In [9]:
negative_df.head(200)

,event,index[],result,event_status,odds,bet,mult,payout,profit
7,knicks_magic_cavs,"[1, 3, 5]","[1, 1, 1]",True,303.388502,0.010,4.033885,0.0403,-0.282
6,knicks_magic_hawks,"[1, 3, 4]","[1, 1, 0]",False,469.572105,0.038,5.695721,0.2148,-0.108
3,wizards_magic_cavs,"[0, 3, 5]","[0, 1, 1]",False,471.712950,0.038,5.717129,0.2171,-0.105


In [10]:
positive_df.head(200)

,event,index[],result,event_status,odds,bet,mult,payout,profit
0,wizards_bulls_hawks,"[0, 2, 4]","[0, 0, 0]",False,1303.368000,0.034,14.033680,0.4725,0.150
1,wizards_bulls_cavs,"[0, 2, 5]","[0, 0, 1]",False,893.908428,0.048,9.939084,0.4725,0.150
4,knicks_bulls_hawks,"[1, 2, 4]","[1, 0, 0]",False,890.186624,0.048,9.901866,0.4725,0.150
2,wizards_magic_hawks,"[0, 3, 4]","[0, 1, 0]",False,707.241026,0.058,8.072410,0.4643,0.142
5,knicks_bulls_cavs,"[1, 2, 5]","[1, 0, 1]",False,601.280655,0.050,7.012807,0.3531,0.031


In [11]:
positive_df.groupby('event_status').count()

,event,index[],result,odds,bet,mult,payout,profit
event_status,,,,,,,,
False,5,5,5,5,5,5,5,5


In [12]:
negative_df.groupby('event_status').count()

,event,index[],result,odds,bet,mult,payout,profit
event_status,,,,,,,,
False,2,2,2,2,2,2,2,2
True,1,1,1,1,1,1,1,1


In [13]:
neg = negative_df.reset_index(drop=True).style.applymap(color_negative_red)
neg

,event,index[],result,event_status,odds,bet,mult,payout,profit
0,knicks_magic_cavs,"['1', '3', '5']","[1, 1, 1]",True,303.389,0.01,4.03389,0.0403,-0.282
1,knicks_magic_hawks,"['1', '3', '4']","[1, 1, 0]",False,469.572,0.038,5.69572,0.2148,-0.108
2,wizards_magic_cavs,"['0', '3', '5']","[0, 1, 1]",False,471.713,0.038,5.71713,0.2171,-0.105


In [14]:
pos = positive_df.reset_index(drop=True).style.applymap(color_positive_green)
pos

,event,index[],result,event_status,odds,bet,mult,payout,profit
0,wizards_bulls_hawks,"['0', '2', '4']","[0, 0, 0]",False,1303.37,0.034,14.0337,0.4725,0.15
1,wizards_bulls_cavs,"['0', '2', '5']","[0, 0, 1]",False,893.908,0.048,9.93908,0.4725,0.15
2,knicks_bulls_hawks,"['1', '2', '4']","[1, 0, 0]",False,890.187,0.048,9.90187,0.4725,0.15
3,wizards_magic_hawks,"['0', '3', '4']","[0, 1, 0]",False,707.241,0.058,8.07241,0.4643,0.142
4,knicks_bulls_cavs,"['1', '2', '5']","[1, 0, 1]",False,601.281,0.05,7.01281,0.3531,0.031


In [15]:
sf49ers = MoneyLine(event="parlay", bet_amount=100, odds=1002)
sf49ers.print_stats()

    event  bet_amount  odds  multiplier  payout
0  parlay         100  1002       11.02  1102.0


In [16]:
pos = MoneyLine(event="pos", bet_amount=100, odds=-1020)
pos.print_stats()

  event  bet_amount  odds  multiplier      payout
0   pos         100 -1020    1.098039  109.803922


In [17]:
neg = MoneyLine(event="neg", bet_amount=1.0, odds=-305)
neg.print_stats()

  event  bet_amount  odds  multiplier    payout
0   neg         1.0  -305    1.327869  1.327869


In [18]:
neg = MoneyLine(event="neg", bet_amount=1.0, odds=-285)
neg.print_stats()

  event  bet_amount  odds  multiplier    payout
0   neg         1.0  -285    1.350877  1.350877
